In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as data
import os
import torch
import torchvision
import cv2
import gdown
import zipfile
import torchvision.transforms as transforms
import random
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
import shutil

# Path to the folder you want to delete
folder_path = '/content/data'
folder_path2 = '/content/reslt_3hidden_layers_test_2'

# Delete the folder
shutil.rmtree(folder_path)
shutil.rmtree(folder_path2)

print(f"Folder '{folder_path}' has been deleted.")

#  Device and parameters


In [ ]:

device = torch.device("cpu")
N = 128  # Batch size
log_interval = 150  # Plotting frequency
num_epoch = 10  # Number of epochs
size_data_train = 4000

lr = 1e-3  # Learning rate
reg_w_decay = 1e-2  # Weight decay
args_dico = {
    'name': 'reslt_3hidden_layers_old_test',
    'save_dir': './reslt_3hidden_layers_test_2',
}

args_dico2 = args_dico.copy()
seed = 2

# Set random seeds for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Create results directory
os.makedirs(args_dico['save_dir'], exist_ok=True)
print(f"Directory {args_dico['save_dir']} is ready.")


In [ ]:


class DatasetFromFolder(data.Dataset):

    def __init__(self, data_feature, data_target,transform=None,phase='train'):
        self.data_feature = data_feature
        self.data_target = data_target
        self.transform =transform
        self.phase=phase

    def __len__(self):
        return len(self.data_feature)

    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)

        data_feature = (self.data_feature[index])
        data_target = (self.data_target[index])

        if self.transform:
            sample = self.transform(data_feature)

        return sample,data_target


        return data_feature,data_target

In [ ]:

#parameter of the DNN
D_in=784#(28X28 images)
H1=200 # number of neurons first hidden layer
H2=200 # number of neurons second hidden layer
H3=200 # number of neurons third hidden layer
nb_class=10 # number of classes
nb_class=10
nb_models=20

# Transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])




# MNIST Dataset

In [ ]:

trainset = torchvision.datasets.MNIST('./data', download=True, train=True, transform=transform)
testset = torchvision.datasets.MNIST('./data', download=True, train=False, transform=transform)

trainloader_mnist = torch.utils.data.DataLoader(trainset, batch_size=N, shuffle=True, num_workers=0)
testloader_mnist = torch.utAils.data.DataLoader(testset, batch_size=N, shuffle=False, num_workers=0)


# Loading NotMNIST Dataset

In [ ]:

folders_notMNIST = os.listdir('/content/drive/MyDrive/BAYESIAN_DNN/base/notMNIST')
NotMNIST_x_list, NotMNIST_y_list_onehot, NotMNIST_y_list = [], [], []

for idx, folder in enumerate(folders_notMNIST):
    files_notMNIST = os.listdir(f'/content/drive/MyDrive/Bayesian_1/notMNIST/{folder}')
    for file in files_notMNIST:
        img_NotMNIST = cv2.imread(f'/content/drive/MyDrive/Bayesian_1/notMNIST/{folder}/{file}', 0)
        NotMNIST_x_list.append(img_NotMNIST)
        label_temp = np.zeros([10])
        label_temp[idx] = 1
        NotMNIST_y_list_onehot.append(label_temp)
        NotMNIST_y_list.append(idx)

NotMNIST_x = np.stack(NotMNIST_x_list, axis=0)
NotMNIST_y = np.asarray(np.stack(NotMNIST_y_list, axis=0), np.int64)
print(f"NotMNIST X shape: {NotMNIST_x.shape}")
print(f"NotMNIST Y shape: {NotMNIST_y.shape}")

trainloader_NOTmnist = torch.utils.data.DataLoader(
    DatasetFromFolder(NotMNIST_x, NotMNIST_y, transform=transform, phase='test'),
    batch_size=N, shuffle=False, num_workers=0)

In [ ]:
#collects the name of all Batch Normalization
def list_bn_fc(model):
    list_bn=[]
    for name, module1 in model.named_modules():

         if  isinstance(module1, torch.nn.modules.batchnorm._BatchNorm):
             list_bn.append(name)

    return list_bn

# Transform the parameter in model_kalman as per model
def prepare_mu_model(model, model_kalman):
    # Get the state_dict from the Kalman model and list of batch normalization layers
    state_dict = model_kalman.state_dict()
    list_bn = list_bn_fc(model)

    # Update the state_dict based on whether the parameter belongs to a BatchNorm layer or not
    for name, param in state_dict.items():
        if any(name_bn in name for name_bn in list_bn):
            transformed_param = param
        else:
            transformed_param = torch.zeros_like(param)


        state_dict[name] = transformed_param

    # Load the updated state_dict into the model_kalman
    model_kalman.load_state_dict(state_dict)

    return model_kalman





def prepare_var_model(model, model_kalman):
    # Get the list of BatchNorm layers and the state_dict from the Kalman model
    list_bn = list_bn_fc(model)
    state_dict = model_kalman.state_dict()

    # Update the state_dict based on whether the parameter belongs to a BatchNorm layer or not
    for name, param in state_dict.items():
        if any(name_bn in name for name_bn in list_bn):
            transformed_param = param
        else:
            transformed_param = torch.ones_like(param) * np.sqrt(2 / param.size(0))

        # Update the parameter in the state_dict
        state_dict[name] = transformed_param


    model_kalman.load_state_dict(state_dict)

    return model_kalman

In [ ]:
#reshaping the input tensor in the forward pass
class hiddenlayers2(torch.nn.Module):
    def __init__(self, D_in,H1,H2,H3, D_out,p=0.5):

        super(hiddenlayers2, self).__init__()
        self.D_in=D_in
        self.H1 = H1
        self.H2 = H2
        self.H3 = H3
        self.D_out = D_out

        self.fc1 = torch.nn.Linear(self.D_in, self.H1)#,bias=False)
        self.bn1 = nn.BatchNorm1d(num_features=H1)
        self.fc2 = torch.nn.Linear(self.H1, self.H2)#,bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=H2)
        self.fc3 = torch.nn.Linear(self.H2, self.H3)#,bias=False)
        self.bn3 = nn.BatchNorm1d(num_features=H3)
        self.fpred = torch.nn.Linear(self.H3, self.D_out)#,bias=False)
        self.p = p
        torch.nn.init.kaiming_normal_(self.fc1.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fc2.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fc3.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fpred.weight, a=0, mode='fan_in', nonlinearity='relu')

        nn.init.constant_(self.fc1.bias,0)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.constant_(self.fc3.bias, 0)
        nn.init.constant_(self.fpred.bias, 0)
    def forward(self, x):

        x = x.view(x.shape[0], -1) # This reshape is needed for the BN update
        h1 = F.relu(self.bn1(self.fc1(x)))
        h_relu = F.relu(self.bn2(self.fc2(h1)))
        h_relu = F.dropout(h_relu, p=self.p, training=self.training)
        h_relu = F.relu(self.bn3(self.fc3(h_relu)))
        h_relu = F.dropout(h_relu, p=self.p, training=self.training)
        y_pred = self.fpred(h_relu)
        return y_pred


class hiddenlayers(torch.nn.Module):
    def __init__(self, D_in,H1,H2,H3, D_out,p=0.5):

        super(hiddenlayers, self).__init__()
        self.D_in=D_in
        self.H1 = H1
        self.H2 = H2
        self.H3 = H3
        self.D_out = D_out

        self.fc1 = torch.nn.Linear(self.D_in, self.H1)#,bias=False)
        self.bn1 = nn.BatchNorm1d(num_features=H1)
        self.fc2 = torch.nn.Linear(self.H1, self.H2)#,bias=False)
        self.bn2 = nn.BatchNorm1d(num_features=H2)
        self.fc3 = torch.nn.Linear(self.H2, self.H3)#,bias=False)
        self.bn3 = nn.BatchNorm1d(num_features=H3)
        self.fpred = torch.nn.Linear(self.H3, self.D_out)#,bias=False)
        self.p = p
        torch.nn.init.kaiming_normal_(self.fc1.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fc2.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fc3.weight, a=0, mode='fan_in', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fpred.weight, a=0, mode='fan_in', nonlinearity='relu')

        nn.init.constant_(self.fc1.bias,0)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.constant_(self.fc3.bias, 0)
        nn.init.constant_(self.fpred.bias, 0)
    def forward(self, x):

        h1 = F.relu(self.bn1(self.fc1(x)))
        h_relu = F.relu(self.bn2(self.fc2(h1)))
        h_relu = F.dropout(h_relu, p=self.p, training=self.training)
        h_relu = F.relu(self.bn3(self.fc3(h_relu)))
        h_relu = F.dropout(h_relu, p=self.p, training=self.training)
        y_pred = self.fpred(h_relu)
        return y_pred







# MODEL INITIALISATION

In [ ]:


# Network Parameters
D_in = 784  # Input size (28x28 images)
H1 = H2 = H3 = 200  # Hidden layers
nb_class = 10  # Number of classes

# Initialize models
net = hiddenlayers(D_in, H1, H2, H3, nb_class, 0.1)
net_mu = hiddenlayers(D_in, H1, H2, H3, nb_class, 0.1)
net_var = hiddenlayers(D_in, H1, H2, H3, nb_class, 0.1)

net_mu = prepare_mu_model(net, net_mu)
net_var = prepare_var_model(net, net_var)
print(f"net_mu and net_variance initialized.")

# Move models to device
net, net_mu, net_var = net.to(device), net_mu.to(device), net_var.to(device)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=reg_w_decay)
lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


In [ ]:
#Functions

#get the current learning rate
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

#update the parameters of models using a Kalman filter
def track_mean_var(net1, net2,net3,optim,n_iter,lendata, PK_mu=0, PK_var=0,epoch_limit=10):

    list_bn = list_bn_fc(net1)
    lr0 = get_lr(optim)
    Q_fc1k_minus_1 = 1e-4 #process noise estimate
    Q_fc1k_minus_1_var = 5e-5

    Pk_fc1_minus = PK_mu + Q_fc1k_minus_1 #prior estimate for mean
    K_kfc1 = Pk_fc1_minus / (Pk_fc1_minus + 1e1 )# kalman gain at step k
    Pk_fc1_minus_var= PK_var + Q_fc1k_minus_1_var #prior estimate for variance
    K_kfc1_var = Pk_fc1_minus_var / (Pk_fc1_minus_var + 1e1)


    state_dict_mu_kalman = net2.state_dict()
    state_dict_var_kalman = net3.state_dict()

    for name, param in net1.named_parameters():


        isbn = False
        for name_bn in list_bn:
            if name_bn in name:
                isbn = True
        if isbn:

            transformed_param=param
            transformed_param_var = param

        else:

            transformed_param = (1 - K_kfc1) * (
                    (net2.state_dict()[name]) - lr0 * param.grad) + K_kfc1 * param

            transformed_param_var = (1 - K_kfc1_var) * ((net3.state_dict()[name]) + (lr0 * param.grad) * (
                    lr0 * param.grad)) + K_kfc1_var * (param - transformed_param) * (param - transformed_param)


        state_dict_mu_kalman[name].copy_(transformed_param)

        state_dict_var_kalman[name].copy_(transformed_param_var)
        PK_mu = (1 - K_kfc1) * Pk_fc1_minus
        PK_var = (1 - K_kfc1_var) * Pk_fc1_minus_var
    net2.load_state_dict(state_dict_mu_kalman)
    net3.load_state_dict(state_dict_var_kalman)
    if n_iter%lendata==1 : print(K_kfc1_var)

    return PK_mu,PK_var



In [ ]:
#TRAINING AND TESTING FUNCTIONS

def train(model, model_mu, model_var, device, train_loader, optimizer, epoch, Pk_fc_mu, Pk_fc_var, log_interval=50):
    model.train()
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        inputs = inputs.view(inputs.shape[0], -1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        n_iter = epoch * len(train_loader) + batch_idx
        with torch.no_grad():
            Pk_fc_mu, Pk_fc_var = track_mean_var(
                model, model_mu, model_var, optimizer, n_iter, len(train_loader), Pk_fc_mu, Pk_fc_var)

        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch}, [{batch_idx * len(inputs)}/{len(train_loader.dataset)}]\tLoss: {loss.item():.6f}")

    lr_scheduler.step()
    return Pk_fc_mu, Pk_fc_var
losses=[]
accuracies=[]
def test( model, device,optimizer, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            inputs = inputs.view(inputs.shape[0], -1)
            output = model(inputs)
            test_loss += criterion(output, target)# F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('CHECK', len(test_loader))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    losses.append(test_loss)
    accuracies.append(accuracy)
    return correct

def plot_loss_accuracy():
    plt.figure(figsize=(10, 5))
    plt.plot(losses, accuracies, marker='o', label='Accuracy vs Loss')
    plt.xlabel('Average Loss')
    plt.ylabel('Accuracy (%)')
    plt.title('Accuracy vs. Average Loss')
    plt.grid(True)
    plt.legend()
    plt.show()


def test_uncertainty( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    BS=0
    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):

            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            #inputs = inputs.view(inputs.shape[0], -1)
            output = model(inputs)
            if batch_idx ==0:
                output_concat = output.clone()
                target_concat = target.clone()
            #print('output_proba',output_proba)
            else:
                output_concat=torch.cat((output_concat, output), 0)
                target_concat=torch.cat((target_concat, target), 0)



    return output_concat , target_concat



In [ ]:
#RETRAINING


def save_checkpoint_kalman(model, model_mu, model_var, optimizer, epoch, miou, args=None,args_dico=None):

    if args!=None:
        name = args.name
        save_dir = args.save_dir
    else:
        name = args_dico['name']
        save_dir = args_dico['save_dir']

    assert os.path.isdir(
        save_dir), "The directory \"{0}\" doesn't exist.".format(save_dir)

    # Save model
    model_path = os.path.join(save_dir, name)
    checkpoint = {
        'epoch': epoch,
        'miou': miou,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'net_mu': model_mu,
        'net_var': model_var,
    }
    torch.save(checkpoint, model_path)
    if args != None:
        # Save arguments
        summary_filename = os.path.join(save_dir, name + '_summary.txt')
        with open(summary_filename, 'w') as summary_file:
            sorted_args = sorted(vars(args))
            summary_file.write("ARGUMENTS\n")
            for arg in sorted_args:
                arg_str = "{0}: {1}\n".format(arg, getattr(args, arg))
                summary_file.write(arg_str)

            summary_file.write("\nBEST VALIDATION\n")
            summary_file.write("Epoch: {0}\n". format(epoch))
            summary_file.write("Mean IoU: {0}\n". format(miou))


Pk_fc_mu, Pk_fc_var = 0, 0
best_correct = 0

for epoch in range(num_epoch):
    Pk_fc_mu, Pk_fc_var = train(net, net_mu, net_var, device, trainloader_mnist, optimizer, epoch, Pk_fc_mu, Pk_fc_var)
    correct = test(net, device, optimizer, testloader_mnist)
    if correct > best_correct:
        save_checkpoint_kalman(net, net_mu, net_var, optimizer, epoch, 0, args_dico=args_dico)
        best_correct = correct
plot_loss_accuracy()

In [ ]:
def load_checkpoint_kalman(model,model_kalman_mu,model_kalman_var, optimizer, folder_dir, filename):

    assert os.path.isdir(
        folder_dir), "The directory \"{0}\" doesn't exist.".format(folder_dir)

    # Create folder to save model and information
    model_path = os.path.join(folder_dir, filename)
    assert os.path.isfile(
        model_path), "The model file \"{0}\" doesn't exist.".format(filename)

    # Load the stored model parameters to the model instance
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch = checkpoint['epoch']
    miou = checkpoint['miou']
    model_kalman_mu.load_state_dict(checkpoint['net_mu'].state_dict())
    model_kalman_var.load_state_dict(checkpoint['net_var'].state_dict())



    return model,model_kalman_mu,model_kalman_var




In [ ]:
#LOAD CHECKPOINTS

# Load the Kalman checkpoint

net, net_mu, net_var = load_checkpoint_kalman(
    net, net_mu, net_var, optimizer, args_dico['save_dir'], args_dico['name']
)

print('The training is finished!')
print('------------------------------------DONE------------------------------------')


In [ ]:
def load_fullnet_kalman(net1, net_mu,net_var,net4, sigma=1, dimfeature=10):

    state_dict = net4.state_dict()
    list_bn=list_bn_fc(net1)
    net4.load_state_dict(net1.state_dict(), strict=True)

    with torch.no_grad():

        for name, param in net1.named_parameters():
            # Transform the parameter as required.
            isbn = False
            for name_bn in list_bn:
                if name_bn in name:
                    isbn = True
            if not isbn:

                # print(name)
                fc_i_weight_0 = np.random.normal(0, 1, (dimfeature, 1))
                var_fc_i_weight = np.sqrt(np.maximum(net_var.state_dict()[name].clone().cpu().data.numpy(), 0))
                # var_fc_i_weight = np.sqrt(1e-4)*np.ones_like(model_var.state_dict()[name].clone().cpu().data.numpy())
                size_data_fc_i_weight = np.prod(np.shape(var_fc_i_weight))
                var_fc_i_weight = np.reshape(var_fc_i_weight, (size_data_fc_i_weight, 1))

                Z_approx_i_FC = np.multiply(var_fc_i_weight, np.cos(np.multiply(
                    np.reshape(net1.state_dict()[name].clone().cpu().data.numpy(), (size_data_fc_i_weight, 1)),
                    np.random.normal(0, sigma, (1, dimfeature))) + np.random.uniform(0, 6.28318, (
                    size_data_fc_i_weight, dimfeature))))
                # print('name',np.mean(Z_approx_i_FC)*np.sqrt(2 / (dimfeature)))
                fc_i_weight_numpy0 = np.reshape(net_mu.state_dict()[name].clone().cpu().data.numpy(),
                                                (size_data_fc_i_weight, 1)) + np.sqrt(2 / (dimfeature)) * np.matmul(Z_approx_i_FC, fc_i_weight_0)

                fc_i_weight_numpy = fc_i_weight_numpy0.astype(np.float32)
                del (fc_i_weight_numpy0)
                fc_i_weight_numpy = np.reshape(fc_i_weight_numpy,
                                               np.shape(net_mu.state_dict()[name].clone().cpu().data.numpy()))
                transformed_param = torch.tensor(fc_i_weight_numpy)
                state_dict[name].copy_(transformed_param)



        net4.load_state_dict(state_dict)
    return net4

In [ ]:
def _check_bn_apply(module, flag):
    if issubclass(module.__class__, torch.nn.modules.batchnorm._BatchNorm):
        flag[0] = True


def _check_bn(model):
    flag = [False]
    model.apply(lambda module: _check_bn_apply(module, flag))
    return flag[0]


def _reset_bn(module):
    if issubclass(module.__class__, torch.nn.modules.batchnorm._BatchNorm):
        module.running_mean = torch.zeros_like(module.running_mean)
        module.running_var = torch.ones_like(module.running_var)


def _get_momenta(module, momenta):
    if issubclass(module.__class__, torch.nn.modules.batchnorm._BatchNorm):
        momenta[module] = module.momentum


def _set_momenta(module, momenta):
    if issubclass(module.__class__, torch.nn.modules.batchnorm._BatchNorm):
        module.momentum = momenta[module]


def bn_update(loader, model, device=None):

    if not _check_bn(model):
        return
    was_training = model.training
    model.train()
    momenta = {}
    model.apply(_reset_bn)
    model.apply(lambda module: _get_momenta(module, momenta))
    n = 0
    for input in loader:
        if isinstance(input, (list, tuple)):
            input = input[0]
        b = input.size(0)

        momentum = b / float(n + b)
        for module in momenta.keys():
            module.momentum = momentum

        if device is not None:
            input = input.to(device)

        model(input)
        n += b

    model.apply(lambda module: _set_momenta(module, momenta))
    model.train(was_training)


#EVALUATION of NOTMNIST

In [ ]:


print('EVALUATION NOTMNIST')

output = torch.zeros(len(trainloader_NOTmnist.dataset), nb_class).to(device)
m = torch.nn.Softmax(dim=1)

for step in range(nb_models):
    net4 = hiddenlayers2(D_in, H1, H2, H3, nb_class, 0.1).to(device)
    net4 = load_fullnet_kalman(net, net_mu, net_var, net4, sigma=1, dimfeature=10)
    bn_update(trainloader_mnist, net4, 'cpu')

    output_temp, target = test_uncertainty(net4, device, trainloader_NOTmnist)
    output += output_temp

output = output / nb_models
NLL = criterion(output, target).item()
output_proba = m(output)

pred = output.argmax(dim=1, keepdim=True)
target_onehot = torch.nn.functional.one_hot(target, nb_class)

BS = ((target_onehot.type(torch.FloatTensor) - output_proba) ** 2).sum().item()
correct = pred.eq(target.view_as(pred)).sum().item()

scores, _ = output_proba.max(1)
scores = scores.view(-1)
scores_notmnits = scores.clone()
labels0 = target.view(-1)
pred0 = pred.view(-1)

output_proba_score0 = output_proba.clone().cpu().data.numpy()
output_proba_score_NOTMNIST = np.amax(output_proba_score0, axis=1, keepdims=True)

# Save Histogram for NOTMNIST
plt.hist(output_proba_score_NOTMNIST, bins=np.linspace(0, 1, 11))
plt.title("Histogram of NOTMNIST (OOD)")
plt.savefig('Histogram_NOT_MNIST_tradi.png')
plt.close()

print('--------------------------------------DONE------------------------------------')


#EVALUATION: MNIST

In [ ]:

print('EVALUATION MNIST (ID)')

output = torch.zeros(len(testloader_mnist.dataset), nb_class).to(device)
m = torch.nn.Softmax(dim=1)

for step in range(nb_models):
    net4 = hiddenlayers2(D_in, H1, H2, H3, nb_class, 0.1).to(device)
    net4 = load_fullnet_kalman(net, net_mu, net_var, net4, sigma=1, dimfeature=10)
    bn_update(trainloader_mnist, net4, 'cpu')

    output_temp, target = test_uncertainty(net4, device, testloader_mnist)
    output += output_temp

output = output / nb_models
NLL = criterion(output, target).item()
output_proba = m(output)

pred = output.argmax(dim=1, keepdim=True)
target_onehot = torch.nn.functional.one_hot(target, nb_class)

BS = ((target_onehot.type(torch.FloatTensor) - output_proba) ** 2).sum().item()
correct = pred.eq(target.view_as(pred)).sum().item()

scores, _ = output_proba.max(1)
scores = scores.view(-1)
labels0 = target.view(-1)
pred0 = pred.view(-1)

# EVALUATION OF THE OOD
conf = torch.cat((scores_notmnits, scores), 0)
label = torch.cat((10 * torch.ones_like(scores_notmnits.view(-1)).long(), labels0.long()), 0)
pred = torch.cat((0 * torch.ones_like(scores_notmnits.view(-1)).long(), pred0.long()), 0)

# Save Histogram for MNIST
output_proba_score0 = output_proba.clone().cpu().data.numpy()
output_proba_score_MNIST = np.amax(output_proba_score0, axis=1, keepdims=True) #maximum confidence score

plt.hist(output_proba_score_MNIST, bins=np.linspace(0, 1, 11))
plt.title("Histogram of MNIST (ID)")
plt.savefig('Histogram_MNIST.png')
plt.close()

print(f'Correct: {correct / len(testloader_mnist.dataset):.4f}')
print(f'Error: {1 - correct / len(testloader_mnist.dataset):.4f}')
print(f'BS: {BS / len(testloader_mnist.dataset):.4f}')
print(f'NLL: {NLL:.4f}')
print('--------------------------------------DONE------------------------------------')


EVALUATION MNIST (ID)
Correct: 0.9816
Error: 0.0184
BS: 0.0310
NLL: 0.0893
--------------------------------------DONE------------------------------------
